In [10]:
# Install required packages
!apt-get install -y poppler-utils tesseract-ocr
!pip install -U layoutparser pdf2image torch torchvision
!pip install -U layoutparser[ocr]
!pip install 'git+https://github.com/facebookresearch/detectron2.git@v0.4#egg=detectron2'
!pip install Pillow==9.5.0
!pip install faiss-gpu faiss-cpu
!pip install   opencv-python numpy   pytesseract faiss-cpu sentence-transformers pillow google-colab

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
tesseract-ocr is already the newest version (4.1.1-2.1build1).
poppler-utils is already the newest version (22.02.0-2ubuntu0.6).
0 upgraded, 0 newly installed, 0 to remove and 21 not upgraded.
  Cloning https://github.com/facebookresearch/detectron2.git (to revision v0.4) to /tmp/pip-install-qad1bett/detectron2_a28dcb12e3084b849a4798835728e03a
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-install-qad1bett/detectron2_a28dcb12e3084b849a4798835728e03a
  Running command git checkout -q 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Resolved https://github.com/facebookresearch/detectron2.git to commit 4aca4bdaa9ad48b8e91d7520e0d0815bb8ca0fb1
  Preparing metadata (setup.py) ... done
ERROR: Could not find a version that satisfies the requirement faiss-gpu (from versions: none)
ERROR: No matching distribution found for faiss-gpu
   ━━

In [14]:
# Import additional libraries
import os
from google.colab import drive
import layoutparser as lp
import cv2
import numpy as np
from pdf2image import convert_from_path
import torch
import shutil
from PIL import Image
import pytesseract
import faiss
from sentence_transformers import SentenceTransformer
import pickle
from datetime import datetime

# Mount Google Drive
drive.mount('/content/drive', force_remount=True)

# Set paths
base_dir = '/content/drive/MyDrive/lifesciences'
pdf_dir = os.path.join(base_dir, 'training_documents')
output_dir = os.path.join(base_dir, 'processed_images')
intermediate_dir = os.path.join(base_dir, 'intermediate_outputs')
faiss_dir = os.path.join(base_dir, 'faiss_index')
timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')

# Create directories if they don't exist
for directory in [output_dir, intermediate_dir, faiss_dir]:
    os.makedirs(directory, exist_ok=True)
    print(f"Directory created/confirmed: {directory}")

def load_layout_model():
    """Load the model using LayoutParser's default mechanism"""
    model = lp.Detectron2LayoutModel('lp://PubLayNet/faster_rcnn_R_50_FPN_3x/config',
                                    extra_config=["MODEL.ROI_HEADS.SCORE_THRESH_TEST", 0.8],
                                    label_map={0: "Text", 1: "Title", 2: "List", 3:"Table", 4:"Figure"})
    return model

def convert_pdf_to_images(pdf_path, output_dir):
    """Convert PDF to images and return list of image paths"""
    try:
        images = convert_from_path(pdf_path, dpi=200)
        image_paths = []
        for i, image in enumerate(images):
            output_path = os.path.join(output_dir, f'{os.path.basename(pdf_path)}_page_{i}_{timestamp}.jpg')
            image.save(output_path, 'JPEG')
            image_paths.append(output_path)
            print(f"Saved image: {output_path}")
        return image_paths
    except Exception as e:
        print(f"Error converting {pdf_path}: {str(e)}")
        return []

def detect_layout(image_path):
    """Detect layout elements in an image"""
    image = cv2.imread(image_path)
    image_rgb = image[..., ::-1]  # Convert BGR to RGB
    layout = model.detect(image_rgb)
    return image, layout

def extract_text(image, coordinates):
    """Extract text from a specific region of the image"""
    x1, y1, x2, y2 = map(int, coordinates)
    roi = image[y1:y2, x1:x2]
    roi_rgb = cv2.cvtColor(roi, cv2.COLOR_BGR2RGB)
    text = pytesseract.image_to_string(roi_rgb, lang='eng')
    return text.strip()

def group_content_by_sections(image, layout):
    """Group content into sections based on titles with actual text"""
    sections = {}
    current_section = "No Section"

    sorted_layout = sorted(layout, key=lambda x: x.coordinates[1])

    for element in sorted_layout:
        element_type = element.type
        coordinates = element.coordinates

        if element_type == "Title":
            text_content = extract_text(image, coordinates)
            current_section = text_content if text_content else f"Section_{coordinates[1]}"
            sections[current_section] = {
                "tables": [], "lists": [], "paragraphs": []
            }
        elif element_type == "Table":
            text_content = extract_text(image, coordinates)
            sections.setdefault(current_section, {"tables": [], "lists": [], "paragraphs": []})
            sections[current_section]["tables"].append({
                "coordinates": coordinates,
                "confidence": element.score,
                "content": text_content
            })
        elif element_type == "List":
            text_content = extract_text(image, coordinates)
            sections.setdefault(current_section, {"tables": [], "lists": [], "paragraphs": []})
            sections[current_section]["lists"].append({
                "coordinates": coordinates,
                "confidence": element.score,
                "content": text_content
            })
        elif element_type == "Text":
            text_content = extract_text(image, coordinates)
            sections.setdefault(current_section, {"tables": [], "lists": [], "paragraphs": []})
            sections[current_section]["paragraphs"].append({
                "coordinates": coordinates,
                "confidence": element.score,
                "content": text_content
            })

    return sections

def save_intermediate_results(pdf_name, page_data, intermediate_dir):
    """Save intermediate results to Google Drive"""
    output_file = os.path.join(intermediate_dir, f'{pdf_name}_page_{page_data["page_number"]}_{timestamp}.pkl')
    with open(output_file, 'wb') as f:
        pickle.dump(page_data, f)
    print(f"Saved intermediate result: {output_file}")
    return output_file

def process_pdfs(pdf_dir, output_dir, intermediate_dir):
    """Process all PDFs in the directory and save intermediates"""
    results = {}

    for pdf_file in os.listdir(pdf_dir):
        if pdf_file.lower().endswith('.pdf'):
            pdf_path = os.path.join(pdf_dir, pdf_file)
            print(f"Processing: {pdf_file}")

            image_paths = convert_pdf_to_images(pdf_path, output_dir)

            pdf_results = []
            for i, img_path in enumerate(image_paths):
                image, layout = detect_layout(img_path)
                sections = group_content_by_sections(image, layout)
                page_data = {
                    "page_number": i,
                    "page_image": img_path,
                    "sections": sections
                }
                pdf_results.append(page_data)
                save_intermediate_results(pdf_file, page_data, intermediate_dir)

            results[pdf_file] = pdf_results

    return results

def create_faiss_index(processed_results, faiss_dir):
    """Create and save a FAISS index from processed document contents"""
    embedder = SentenceTransformer('all-MiniLM-L6-v2')
    all_texts = []
    metadata = []

    for pdf_name, pages in processed_results.items():
        for page in pages:
            for section_name, content in page["sections"].items():
                for table in content["tables"]:
                    all_texts.append(table["content"])
                    metadata.append({"pdf": pdf_name, "page": page["page_number"], "section": section_name, "type": "table"})
                for lst in content["lists"]:
                    all_texts.append(lst["content"])
                    metadata.append({"pdf": pdf_name, "page": page["page_number"], "section": section_name, "type": "list"})
                for para in content["paragraphs"]:
                    all_texts.append(para["content"])
                    metadata.append({"pdf": pdf_name, "page": page["page_number"], "section": section_name, "type": "paragraph"})

    # Generate embeddings
    embeddings = embedder.encode(all_texts, convert_to_numpy=True)

    # Create FAISS index
    dimension = embeddings.shape[1]
    index = faiss.IndexFlatL2(dimension)
    index.add(embeddings)

    # Save FAISS index and metadata
    faiss_index_path = os.path.join(faiss_dir, f'faiss_index_{timestamp}.index')
    metadata_path = os.path.join(faiss_dir, f'metadata_{timestamp}.pkl')

    faiss.write_index(index, faiss_index_path)
    with open(metadata_path, 'wb') as f:
        pickle.dump(metadata, f)

    print(f"Saved FAISS index: {faiss_index_path}")
    print(f"Saved metadata: {metadata_path}")

    return index, metadata, embedder

def query_faiss_index(query, index, metadata, embedder, processed_results, k=5):
    """Query the FAISS index, identify the most relevant section, and prioritize its content"""
    # Step 1: Encode the query
    query_embedding = embedder.encode([query], convert_to_numpy=True)

    # Step 2: Extract all section titles from processed_results
    section_titles = []
    section_metadata = []
    for pdf_name, pages in processed_results.items():
        for page in pages:
            for section_name in page["sections"].keys():
                if section_name != "No Section":  # Skip placeholder sections
                    section_titles.append(section_name)
                    section_metadata.append({"pdf": pdf_name, "page": page["page_number"], "section": section_name})

    # Step 3: Find the most relevant section title
    section_embeddings = embedder.encode(section_titles, convert_to_numpy=True)
    section_distances, section_indices = index.search(query_embedding, 1)  # Get top 1 section
    most_relevant_section = section_titles[section_indices[0][0]]
    most_relevant_section_metadata = section_metadata[section_indices[0][0]]
    print(f"Most relevant section identified: {most_relevant_section}")

    # Step 4: Perform FAISS search across all content
    distances, indices = index.search(query_embedding, k * 2)  # Get more results to filter

    # Step 5: Collect and prioritize results
    prioritized_results = []
    other_results = []

    for i, idx in enumerate(indices[0]):
        meta = metadata[idx]
        content_key = f"{meta['type']}s"
        content = processed_results[meta["pdf"]][meta["page"]]["sections"][meta["section"]][content_key][0]["content"]
        result = {
            "distance": distances[0][i],
            "metadata": meta,
            "content": content
        }

        # Prioritize results from the most relevant section
        if meta["section"] == most_relevant_section:
            prioritized_results.append(result)
        else:
            other_results.append(result)

    # Step 6: Combine results, prioritizing the relevant section, up to k total
    final_results = prioritized_results + other_results
    final_results = final_results[:k]

    return final_results

# Example usage with your existing variables
query = "What does the document say about General Requirements?"
results = query_faiss_index(query, faiss_index, metadata, embedder, processed_results, k=5)

# Display query results
print("\nQuery Results (prioritized by relevant section):")
for result in results:
    print(f"Distance: {result['distance']:.4f}")
    print(f"Metadata: {result['metadata']}")
    print(f"Content: {result['content'][:200]}...")  # Truncate for brevity
    print()


# Load model
model = load_layout_model()
if model is None:
    raise Exception("Failed to load layout model")

# Process the PDFs
processed_results = process_pdfs(pdf_dir, output_dir, intermediate_dir)

# Create and save FAISS index
faiss_index, metadata, embedder = create_faiss_index(processed_results, faiss_dir)

# Example query with the fixed call
query = "What does the document say about General Requirements?"
results = query_faiss_index(query, faiss_index, metadata, embedder, processed_results, k=5)

# Display query results
print("\nQuery Results (prioritized by relevant section):")
for result in results:
    print(f"Distance: {result['distance']:.4f}")
    print(f"Metadata: {result['metadata']}")
    print(f"Content: {result['content'][:200]}...")  # Truncate for brevity
    print()

Mounted at /content/drive
Directory created/confirmed: /content/drive/MyDrive/lifesciences/processed_images
Directory created/confirmed: /content/drive/MyDrive/lifesciences/intermediate_outputs
Directory created/confirmed: /content/drive/MyDrive/lifesciences/faiss_index
Most relevant section identified: 1.8 Vendor and Warranty Specifications

Query Results (prioritized by relevant section):
Distance: 1.1667
Metadata: {'pdf': 'ProtonGlow_URS.pdf', 'page': 3, 'section': '2.20 Documentation Requirements', 'type': 'paragraph'}
Content: 2.21 Training Requirements...

Distance: 1.2369
Metadata: {'pdf': 'ProtonGlow_URS.pdf', 'page': 1, 'section': 'No Section', 'type': 'list'}
Content: The system must adhere to all relevant compliance standards applicable to
pharmaceutical manufacturing, including GMP and FDA regulations.

 

 

 

 

 

 

 

1.2 | The system must achieve a minimum...

Distance: 1.3082
Metadata: {'pdf': 'ProtonGlow_URS.pdf', 'page': 5, 'section': '4 Process Safety', 'type': '